In [1]:
## package setup and import
import numpy as np
import datajoint as dj
import pandas as pd
import matplotlib.pyplot as plt
import os
import datetime
from pathlib import Path
from scipy.io import loadmat

In [2]:
## datajoint setup/import for database management
dj.__version__

dj.config['database.host'] = '127.0.0.1'
dj.config['database.port'] = 3306
dj.config['database.user'] = 'root'
dj.config['database.password'] = 'simple'
dj.config['fetch_format'] = 'array'
dj.config['enable_python_native_blobs'] = True 


connection = dj.conn()
connection

Connecting root@127.0.0.1:3306


DataJoint connection (connected) root@127.0.0.1:3306

In [3]:
## SETTING UP DATA STORES

datastorefolder = 'D:\Dropbox\BpodInfoseek\Data\''

dj.config['stores'] = {
    'raw_data':{
        'protocol':'file',
        'location': os.path.join(datastorefolder,'raw_data')
    }
}

dj.config.save_global()

In [4]:
## ESTABLISH SCHEMA

schema = dj.schema('simple')
schema.drop(True)
schema = dj.schema('simple')

In [5]:
## Manual entry table for mouse identification/attribute labelling
@schema
class Mice(dj.Manual): ## Manual entry table for mouse identification/attribute labelling
    definition = """
    mouse_id : varchar(100) 
    ---
    sex = 'U': enum('F','M','U')
    
    geno = 'C57BL6' : varchar(50)
    """

#     DOB = null : date  # YYYY-MM-DD
#     imaging = null : int

# format for entry of new mouse with new attributes, load later on will create new mouse if not previously entered but will be missing experimental params
mice = Mice()

mice.insert([{'mouse_id' : 'JB366', 'sex' : 'M'},{'mouse_id' : 'JB367','sex' :'F'},{'mouse_id' : 'JB368', 'sex' : 'M'},{'mouse_id' : 'JB369', 'sex' : 'M'}],skip_duplicates=True)
mice

# dj.ERD(schema)

mouse_id,sex,geno
JB366,M,C57BL6
JB367,F,C57BL6
JB368,M,C57BL6
JB369,M,C57BL6


In [6]:
mice.proj('sex')

mouse_id,sex
JB366,M
JB367,F
JB368,M
JB369,M


In [7]:
# List all data files in directory using pathlib for insertion in manual table

files = []
datafolder = Path('D:\Dropbox\BpodInfoseek\Data\PythonTest')
# datafolder = Path(r'C:\Users\jbuss\Dropbox\BpodInfoseek\Data\PythonTest')
for file in datafolder.glob('JB*.mat'):
    if file.stat().st_size>3000: # leaves out garbage files!
        thisfile = {}
        thisfile["filename"] = file.name
        thisfile["filepath"] = str(file.parent)
        mouseID,protocol,datestr,timestr = file.stem.split('_')
        thisfile["mouse_id"] = mouseID
        thisfile["protocol"] = protocol
        date_obj = datetime.datetime.strptime(datestr, '%Y%m%d')
        thisfile["sessiondate"] = date_obj.date()
        thisfile["datestr"] = datestr
        time_obj = datetime.datetime.strptime(timestr, '%H%M%S')
#         time_obj = datetime.datetime.strptime(timestr[:len(timestr)-4], '%H%M%S')
        thisfile["sessiontime"] = time_obj.time()
        thisfile["timestr"] = timestr
        files.append(thisfile)

In [8]:
files

[{'filename': 'JB366_Infoseek_20201013_102926.mat',
  'filepath': 'D:\\Dropbox\\BpodInfoseek\\Data\\PythonTest',
  'mouse_id': 'JB366',
  'protocol': 'Infoseek',
  'sessiondate': datetime.date(2020, 10, 13),
  'datestr': '20201013',
  'sessiontime': datetime.time(10, 29, 26),
  'timestr': '102926'},
 {'filename': 'JB366_Infoseek_20201014_103353.mat',
  'filepath': 'D:\\Dropbox\\BpodInfoseek\\Data\\PythonTest',
  'mouse_id': 'JB366',
  'protocol': 'Infoseek',
  'sessiondate': datetime.date(2020, 10, 14),
  'datestr': '20201014',
  'sessiontime': datetime.time(10, 33, 53),
  'timestr': '103353'},
 {'filename': 'JB366_Infoseek_20201014_104211.mat',
  'filepath': 'D:\\Dropbox\\BpodInfoseek\\Data\\PythonTest',
  'mouse_id': 'JB366',
  'protocol': 'Infoseek',
  'sessiondate': datetime.date(2020, 10, 14),
  'datestr': '20201014',
  'sessiontime': datetime.time(10, 42, 11),
  'timestr': '104211'},
 {'filename': 'JB366_Infoseek_20201015_101140.mat',
  'filepath': 'D:\\Dropbox\\BpodInfoseek\\Dat

In [9]:
filesDf = pd.DataFrame(files)
# filesDf.groupby(['mouse_id','date']).sum()
# columns = filesDf[["mouse_id","date"]].values
# mousedays = pd.unique(columns)
# print(mousedays)
mousedaysDf = filesDf.groupby(['mouse_id','sessiondate','datestr']).size().reset_index().rename(columns={0:'sessions'})
# mousedaysDf
mousedaysDfshort = mousedaysDf[['mouse_id','sessiondate','datestr']]
mousedaysDfshort

,mouse_id,sessiondate,datestr
0,JB366,2020-10-13,20201013
1,JB366,2020-10-14,20201014
2,JB366,2020-10-15,20201015
3,JB366,2020-10-16,20201016
4,JB367,2020-10-13,20201013
5,JB367,2020-10-14,20201014
6,JB367,2020-10-15,20201015
7,JB367,2020-10-16,20201016
8,JB368,2020-10-12,20201012
9,JB368,2020-10-13,20201013


In [10]:
# MouseDays.drop(True)
@schema
class MouseDays(dj.Manual):
    definition = """
    ->Mice
    datestr : char(8)
    ---
    sessiondate : date
    """
mouseDays = MouseDays()
mouseDays.insert(mousedaysDfshort)
mouseDays

mouse_id,datestr,sessiondate
JB366,20201013,2020-10-13
JB366,20201014,2020-10-14
JB366,20201015,2020-10-15
JB366,20201016,2020-10-16
JB367,20201013,2020-10-13
JB367,20201014,2020-10-14
JB367,20201015,2020-10-15
JB367,20201016,2020-10-16
JB368,20201012,2020-10-12
JB368,20201013,2020-10-13


In [11]:
# BehaviorSession.drop(True)
@schema
class BehaviorSession(dj.Manual):
    definition = """
    # Infoseek behavior .mat files from a folder
    filename : varchar(127)
    -> MouseDays
    datestr : char(8)
    timestr: char(6)
    protocol : varchar(127)      
    ---
    sessiondate : date
    sessiontime : time
    filepath : varchar(127)
    """
behaviorSession = BehaviorSession()
behaviorSession
    


filename,mouse_id,datestr,timestr,protocol,sessiondate,sessiontime,filepath


In [12]:
behaviorSession.insert(files,skip_duplicates=True)
behaviorSession

filename,mouse_id,datestr,timestr,protocol,sessiondate,sessiontime,filepath
JB366_Infoseek_20201013_102926.mat,JB366,20201013,102926,Infoseek,2020-10-13,10:29:26,D:\Dropbox\BpodInfoseek\Data\PythonTest
JB366_Infoseek_20201014_103353.mat,JB366,20201014,103353,Infoseek,2020-10-14,10:33:53,D:\Dropbox\BpodInfoseek\Data\PythonTest
JB366_Infoseek_20201014_104211.mat,JB366,20201014,104211,Infoseek,2020-10-14,10:42:11,D:\Dropbox\BpodInfoseek\Data\PythonTest
JB366_Infoseek_20201015_101140.mat,JB366,20201015,101140,Infoseek,2020-10-15,10:11:40,D:\Dropbox\BpodInfoseek\Data\PythonTest
JB366_Infoseek_20201016_102403.mat,JB366,20201016,102403,Infoseek,2020-10-16,10:24:03,D:\Dropbox\BpodInfoseek\Data\PythonTest
JB366_Infoseek_20201016_104533.mat,JB366,20201016,104533,Infoseek,2020-10-16,10:45:33,D:\Dropbox\BpodInfoseek\Data\PythonTest
JB367_Infoseek_20201013_112258.mat,JB367,20201013,112258,Infoseek,2020-10-13,11:22:58,D:\Dropbox\BpodInfoseek\Data\PythonTest
JB367_Infoseek_20201014_112858.mat,JB367,20201014,112858,Infoseek,2020-10-14,11:28:58,D:\Dropbox\BpodInfoseek\Data\PythonTest
JB367_Infoseek_20201015_105831.mat,JB367,20201015,105831,Infoseek,2020-10-15,10:58:31,D:\Dropbox\BpodInfoseek\Data\PythonTest
JB367_Infoseek_20201015_111113.mat,JB367,20201015,111113,Infoseek,2020-10-15,11:11:13,D:\Dropbox\BpodInfoseek\Data\PythonTest


In [13]:
len(behaviorSession)

23

In [14]:
filename = r'JB366_TestInfoseek_20200916_113754.mat'
filepath = r'D:\Dropbox\BpodInfoseek\Data\PythonTest'
newpath = Path(filepath,filename)
print(newpath)

D:\Dropbox\BpodInfoseek\Data\PythonTest\JB366_TestInfoseek_20200916_113754.mat


In [15]:
name = 'JB366_TestInfoseek_20200916_113754.mat'
'r'+name
filepath = 'D:\Dropbox\BpodInfoseek\Data\PythonTest'
newpath = Path(filepath,name)
print(newpath)

D:\Dropbox\BpodInfoseek\Data\PythonTest\JB366_TestInfoseek_20200916_113754.mat


In [16]:
# SessionRawData.drop(True)

# Save raw data

@schema
class SessionRawData(dj.Imported):
    definition = """
    -> BehaviorSession
    ---
    sessiondate = "2020-01-01" : date 
    sessiontime = "000000" : time
    raw_data   : longblob     # raw 'session_data' from Bpod matlab file
    """
    def make(self, key):  # key is the primary key of one of the entries in the table `Scan`
        name = (BehaviorSession & key).fetch1()['filename']
        partpath = (BehaviorSession & key).fetch1()['filepath']
        matdata = loadmat(Path(partpath,name))
        key['raw_data'] = pd.DataFrame(matdata['SessionData'][0])
        key['sessiondate'] = (BehaviorSession & key).fetch1()['sessiondate']
        key['sessiontime'] = (BehaviorSession & key).fetch1()['sessiontime']
#         key['raw_data'] = matdata
        self.insert1(key)

In [17]:
SessionRawData.populate()
SessionRawData()

filename,mouse_id,datestr,timestr,protocol,sessiondate,sessiontime,raw_data raw 'session_data' from Bpod matlab file
JB366_Infoseek_20201013_102926.mat,JB366,20201013,102926,Infoseek,2020-10-13,10:29:26,=BLOB=
JB366_Infoseek_20201014_103353.mat,JB366,20201014,103353,Infoseek,2020-10-14,10:33:53,=BLOB=
JB366_Infoseek_20201014_104211.mat,JB366,20201014,104211,Infoseek,2020-10-14,10:42:11,=BLOB=
JB366_Infoseek_20201015_101140.mat,JB366,20201015,101140,Infoseek,2020-10-15,10:11:40,=BLOB=
JB366_Infoseek_20201016_102403.mat,JB366,20201016,102403,Infoseek,2020-10-16,10:24:03,=BLOB=
JB366_Infoseek_20201016_104533.mat,JB366,20201016,104533,Infoseek,2020-10-16,10:45:33,=BLOB=
JB367_Infoseek_20201013_112258.mat,JB367,20201013,112258,Infoseek,2020-10-13,11:22:58,=BLOB=
JB367_Infoseek_20201014_112858.mat,JB367,20201014,112858,Infoseek,2020-10-14,11:28:58,=BLOB=
JB367_Infoseek_20201015_105831.mat,JB367,20201015,105831,Infoseek,2020-10-15,10:58:31,=BLOB=
JB367_Infoseek_20201015_111113.mat,JB367,20201015,111113,Infoseek,2020-10-15,11:11:13,=BLOB=


In [18]:
# ex = SessionRawData & 'filename = "JB368_Infoseek_20201012_162220.mat"'
name = 'JB368_Infoseek_20201012_162220.mat'
filepath = 'D:\Dropbox\BpodInfoseek\Data\PythonTest'
newpath = Path(filepath,name)
ex = loadmat(newpath)
# sorted(ex.keys())
sessiondata = ex['SessionData']
sessiondata.dtype
# data = ex.fetch()['raw_data']
# # data = pd.DataFrame(data)
# data
namelist = sessiondata.dtype.names
for n in sessiondata.dtype.names:
    print(n)

TrialCounts
PlotOutcomes
TrialTypes
Outcomes
OrigTrialTypes
OrigRewardTypes
EventNames
Info
nTrials
RawEvents
RawData
TrialStartTimestamp
TrialEndTimestamp
SettingsFile
TrialSettings
Notes
MarkerCodes


In [19]:
sessiondata = SessionRawData & 'filename = "JB368_Infoseek_20201012_162220.mat"'
data = sessiondata.fetch()['raw_data']

In [26]:
@schema
class Calc(dj.Computed):
    definition = """
    -> SessionRawData
    ---
    """
    def make(self, key):  # key is the primary key of one of the entries in the table `Scan`
        data = (SessionRawData & key).fetch1()['raw_data']
        key['new_data'] = data
#         key['raw_data'] = matdata
        self.insert1(key)

Calc().populate
Calc

__main__.Calc

In [21]:

@schema
class SessionProcessed(dj.Computed):
    definition = """
    -> SessionRawData
    ---
    outcomes : longblob
    """
    
    def make(self, key):
        data = ((SessionRawData & key).fetch1()['raw_data'])
        outcomes = pd.DataFrame(data['Outcomes'][0][0])
        key['outcomes'] = outcomes
        
        self.insert1(key)
        
        print("test")
SessionProcessed.drop(True)        

Tables dropped.  Restart kernel.


In [23]:
SessionProcessed().populate
SessionProcessed

__main__.SessionProcessed

In [ ]:
@schema
class SessionData(dj.Computed):
    definition = """
    -> SessionRawData
    ---
    SessionSettings : blob
    nTrials : longblog
    Outcomes : longblog
    StartTimes : longblog
    EndTimes : longblog
    States : longblog
    Events : longblog
    """
    
    def make(self, key):  # key is the primary key of one of the entries in the table `Scan`
